In [ ]:
import pandas as pd
import re

# Notebook

## Structure:

- 'Invoice Number',        
- 'Date',                     
- 'Due Date',              
- 'Product',               
- 'Order Name',
- 'Description'
- 'Partner Name',
- 'Partner ID',
- 'Advertiser Name',
- 'Advertiser ID',
- 'Campaign Name',
- 'Campaign ID',
- 'Insertion Order Name',
- 'Insertion Order ID',
- 'PO Extract'
- 'Quantity'
- 'UOM',
- 'Amount'            

In [ ]:
meta = pd.read_csv('csv/XXXX.csv', nrows=7)
meta = meta.transpose()

meta.columns = meta.iloc[0] # Set the first row as the header
meta = meta[1:]

meta['Invoice date'] = pd.to_datetime(meta['Invoice date'], dayfirst=False).dt.strftime('%d/%m/%Y')
meta['Due Date'] = pd.to_datetime(meta['Due Date'], dayfirst=False).dt.strftime('%d/%m/%Y')

meta = meta.loc[:, ['Invoice number',
                    'Invoice date', 
                    'Due Date', 
                    'Product']]

In [213]:
data = pd.read_csv(
    'csv/XXXX.csv',
    skiprows=9,
    usecols=['Order name', 'Description', 'Quantity', 'UOM', 'Amount']
)

data.rename(columns={'Description': 'Info'}, inplace=True)  # rename to Info because Description will be charge type
data['Info'] = data['Info'].str.replace('-', '–', n=1, regex=False)
data = data.iloc[:-1]  # remove total row

# Extract details
data['Order name'] = data['Order name'].str.split(':').str[1]
data['Description'] = data['Info'].str.split(' – ').str[0]

# Partner
data['Partner'] = data['Info'].str.split(' – ').str[1]
data['Partner Name'] = data['Partner'].str.split(': ').str[1].str.replace(r'\sID$', '', regex=True)
data['Partner ID'] = data['Partner'].str.split(': ').str[2]

# Advertiser
data['Advertiser'] = data['Info'].str.split(' – ').str[2]
data['Advertiser Name'] = data['Advertiser'].str.split(': ').str[1].str.replace(r'\sID$', '', regex=True)
data['Advertiser ID'] = data['Advertiser'].str.split(': ').str[2]

# Campaign
data['Campaign'] = data['Info'].str.split(' – ').str[3]
data['Campaign Name'] = data['Campaign'].str.split(': ').str[1].str.replace(r'\sID$', '', regex=True)
data['Campaign ID'] = data['Campaign'].str.split(': ').str[2]

# Insertion order
data['Insertion order'] = data['Info'].str.split(' – ').str[4]
data['Insertion order Name'] = data['Insertion order'].str.split(': ').str[1].str.replace(r'\sID$', '', regex=True)
data['Insertion order ID'] = data['Insertion order'].str.split(': ').str[2]

# PO Extract from Campaign Name (last occurrence)
pattern = r'\d{4}\.\d{3}/WPP/[A-Z0-9]+/\d{4}'
def extract_last_po(text):
   matches = re.findall(pattern, str(text))
   return matches[-1] if matches else None

data['PO Extract'] = data['Campaign Name'].apply(extract_last_po)

#Final column order (fixed commas & names)
data = data.loc[:, [
   'Order name',
   'Description',
   'Partner Name',
   'Partner ID',
   'Advertiser Name',
   'Advertiser ID',
   'Campaign Name',
   'Campaign ID',
   'Insertion order Name',
   'Insertion order ID',
   'PO Extract',
   'Quantity',
   'UOM',
   'Amount'
]]


,Order name,Description,Partner Name,Partner ID,Advertiser Name,Advertiser ID,Campaign Name,Campaign ID,Insertion order Name,Insertion order ID,PO Extract,Quantity,UOM,Amount
0,6710291265,Media Cost,Mindshare Indonesia,1365144,BPJS,6710291265,BPJS TK - 2025 03 - Youtube - Video Lebaran_BP...,54805386,BPJS TK - 2025 03 - Youtube - Video Lebaran_BP...,1017981221,1076.001/WPP/BPTKIN/0325,1,EA,"68,357,567.00"
1,6710291265,Platform Fee,Mindshare Indonesia,1365144,BPJS,6710291265,BPJS TK - 2025 03 - Youtube - Video Lebaran_BP...,54805386,BPJS TK - 2025 03 - Youtube - Video Lebaran_BP...,1017981221,1076.001/WPP/BPTKIN/0325,1,EA,"2,187,442.00"


In [172]:
# Merge with meta + data

final = data.copy()
final['Invoice Number'] = meta['Invoice number'].values[0]
final['Date'] = meta['Invoice date'].values[0]
final['Due Date'] = meta['Due Date'].values[0]
final['Product'] = meta['Product'].values[0]

cols = ['Invoice Number', 'Date', 'Due Date', 'Product'] + [c for c in final.columns if c not in ['Invoice Number', 'Date', 'Due Date', 'Product']]
final = final[cols]

# Script

In [168]:
import os

In [174]:
def get_meta(file_path):
    """Extract and format invoice metadata."""
    meta = pd.read_csv(file_path, nrows=7).transpose()
    meta.columns = meta.iloc[0]  # First row = header
    meta = meta[1:]  # Drop header row
    meta = meta.loc[:, ['Invoice number', 'Invoice date', 'Due Date', 'Product']]
    
    # Convert dates to dd/mm/yyyy
    for col in ['Invoice date', 'Due Date']:
        meta[col] = pd.to_datetime(meta[col], dayfirst=False, errors='coerce').dt.strftime('%d/%m/%Y')
    
    return meta

In [214]:
def get_data(file_path):
    """Extract campaign/order data."""
    data = pd.read_csv(
        file_path,
        skiprows=9,
        usecols=['Order name', 'Description', 'Quantity', 'UOM', 'Amount']
    )
    
    # Rename and fix dash
    data.rename(columns={'Description': 'Info'}, inplace=True)
    data['Info'] = data['Info'].str.replace('-', '–', n=1, regex=False)
    data = data.iloc[:-1]  # Remove total row

    # Order name
    data['Order name'] = data['Order name'].str.split(':').str[1]
    data['Description'] = data['Info'].str.split(' – ').str[0]

    # Partner
    data['Partner'] = data['Info'].str.split(' – ').str[1]
    data['Partner Name'] = data['Partner'].str.split(': ').str[1].str.replace(r'\sID$', '', regex=True)
    data['Partner ID'] = data['Partner'].str.split(': ').str[2]

    # Advertiser
    data['Advertiser'] = data['Info'].str.split(' – ').str[2]
    data['Advertiser Name'] = data['Advertiser'].str.split(': ').str[1].str.replace(r'\sID$', '', regex=True)
    data['Advertiser ID'] = data['Advertiser'].str.split(': ').str[2]

    # Campaign
    data['Campaign'] = data['Info'].str.split(' – ').str[3]
    data['Campaign Name'] = data['Campaign'].str.split(': ').str[1].str.replace(r'\sID$', '', regex=True)
    data['Campaign ID'] = data['Campaign'].str.split(': ').str[2]

    # Insertion order
    data['Insertion order'] = data['Info'].str.split(' – ').str[4]
    data['Insertion order Name'] = data['Insertion order'].str.split(': ').str[1].str.replace(r'\sID$', '', regex=True)
    data['Insertion order ID'] = data['Insertion order'].str.split(': ').str[2]

    # PO Extract
    pattern = r'\d{4}\.\d{3}/WPP/[A-Z0-9]+/\d{4}'
    def extract_last_po(text):
        matches = re.findall(pattern, str(text))
        return matches[-1] if matches else None
    data['PO Extract'] = data['Campaign Name'].apply(extract_last_po)

    # Final column order
    data = data.loc[:, [
        'Order name',
        'Description',
        'Partner Name',
        'Partner ID',
        'Advertiser Name',
        'Advertiser ID',
        'Campaign Name',
        'Campaign ID',
        'Insertion order Name',
        'Insertion order ID',
        'PO Extract',
        'Quantity',
        'UOM',
        'Amount'
    ]]
    
    return data

In [215]:
def process_file(file_path):
    """Read one CSV file, process meta and data, and merge."""
    meta = get_meta(file_path)
    data = get_data(file_path)

    final = data.copy()
    final.insert(0, 'Invoice Number', meta['Invoice number'].values[0])
    final.insert(1, 'Date', meta['Invoice date'].values[0])
    final.insert(2, 'Due Date', meta['Due Date'].values[0])
    final.insert(3, 'Product', meta['Product'].values[0])

    return final

In [216]:
for file in os.listdir('csv'):
    if file.endswith('.csv'):
        print(f'Processing {file}...')
        process_file(f'csv/{file}').to_csv(f'clean/{file}', index=False)

Processing 5193246880.csv...
Processing 5217716855.csv...
Processing 5217721589.csv...
Processing 5217722081.csv...
Processing 5217752382.csv...
Processing 5217961198.csv...
Processing 5217967504.csv...
Processing 5217969274.csv...
Processing 5218003753.csv...
Processing 5242557263.csv...
Processing 5242573211.csv...
Processing 5242586334.csv...
Processing 5242589688.csv...
Processing 5242805578.csv...
Processing 5242824316.csv...
Processing 5270911706.csv...
Processing 5270943035.csv...
Processing 5270983232.csv...
Processing 5270988330.csv...
Processing 5270991807.csv...
Processing 5271205711.csv...
Processing 5271244876.csv...
Processing 5271286966.csv...
Processing 5297330708.csv...
Processing 5297341275.csv...
Processing 5297341341.csv...
Processing 5297362247.csv...
Processing 5297362311.csv...
Processing 5297379773.csv...
Processing 5297392812.csv...
Processing 5297602522.csv...
Processing 5297614003.csv...
Processing 5297672740.csv...
Processing 5327972481.csv...
Processing 532

In [217]:
clean_folder = 'clean'
output_file = 'merged.csv'

dfs = []

for file in os.listdir(clean_folder):
    if file.endswith('.csv'):
        df = pd.read_csv(os.path.join(clean_folder, file))
        dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

merged_df.to_csv(output_file, index=False)